## Read a HESTData sample from disk

In [ ]:
from hest import read_HESTData
from hest import load_hest


# Read the whole hest dataset
hest_data = load_hest('hest_data')

# Read a subset of hest
hest_data = load_hest('hest_data', id_list=['TENX96', 'TENX99'])

# or read an individual sample from disk
st = read_HESTData(
    adata_path='my_h5ad.h5ad', # aligned ST counts
    img='my_image.tif', # WSI
    metrics_path='my_metrics.json', # metrics/metadata
    mask_path_pkl='my_mask.pkl', # optional (tissue_mask)
    mask_path_jpg='my_mask.jpg' # optional (tissue_mask)
)

## Visualizing the spots over a full-resolution WSI

In [ ]:
# visualize the spots over a downscaled version of the full resolution image
save_dir = '.'

st.save_spatial_plot(save_dir)

## Saving to pyramidal tiff and h5
Save `HESTData` object to `.tiff` + expression `.h5ad` and a metadata file.

In [ ]:
# Warning saving a large image to pyramidal tiff (>1GB) can be slow on a hard drive.
st.save(save_dir, pyramidal=True)

## Deep learning (or Otsu) based segmentation

In [ ]:
save_dir = '.'
name = 'my_sample'

st.segment_tissue(method='deep') # or method='otsu'
st.save_tissue_seg_pkl(save_dir, name)
st.save_tissue_seg_jpg(save_dir, name)

## Patching

In [ ]:
patch_save_dir = '.'

st.dump_patches(
    patch_save_dir,
    'demo',
    target_patch_size=224,
    target_pixel_size=0.5
)